In [1]:
#basic package
import json
import tqdm
import os
import numpy as np
import glob
import pandas as pd
import sys
import shutil
import time
import datetime as dt

#modelling
from catboost import CatBoostClassifier

In [2]:
PACKAGE_PARENT = '../'
SCRIPT_DIR = os.path.dirname(os.path.realpath(os.path.join(os.getcwd(), os.path.expanduser('__file__'))))
sys.path.append(os.path.normpath(os.path.join(SCRIPT_DIR, PACKAGE_PARENT)))
from UTILS import preprocessing_Origins, cleaning_processing, HenDailyVariable_Origins
import config_origins as config

C:\Users\camil\anaconda3\lib\site-packages\scipy\sparse\sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [3]:
#initialise parameters
path_extracted_data = config.path_extracted_data
path_initial_data = config.path_initial_data
id_run = config.id_run
path_extracted_data_cleaning = os.path.join(path_extracted_data,'Cleaning')
path_extracted_data_cleaning_model = os.path.join(path_extracted_data_cleaning,'model')  
path_extracted_data_cleaning_rawvscleaned = os.path.join(path_extracted_data_cleaning,'Raw&Cleaned')  
#create a director if not existing
if not os.path.exists(path_extracted_data_cleaning_rawvscleaned):
    os.makedirs(path_extracted_data_cleaning_rawvscleaned)
#feature for model
li_cont_select = config.li_cont_select
li_bin = config.li_bin
li_cat_select = config.li_cat_select

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
name_method = 'ML-method'
#name_method = 'TD-method'

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Download data & model

In [5]:
#download data and add the necessary processing as the duration, next/previous zones etc
#steps also done in "1_PreprocessedTraining&TestingData"
date_min = dt.datetime(2020, 11, 13) #when artificial light start to last 15h
date_max = date_min+dt.timedelta(days=60)
START_TIME = time.perf_counter()
df = cleaning_processing(date_min=date_min, date_max=date_max, config=config) #same processing at other sections
df['time'] = df['Timestamp'].map(lambda x: dt.datetime.time(x))
df.rename(columns={'Zone':'Trackingsystem_Zone'}, inplace=True)
df.to_csv(os.path.join(path_extracted_data_cleaning_rawvscleaned,'df_records_initial_processed.csv'), sep=';', index=False)
END_TIME = time.perf_counter()
print ("Total running time: %.2f mn" %((END_TIME-START_TIME)/60))
print(df.shape)
df.head(3)

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
C:\Users\camil\anaconda3\lib\site-packages\numpy\lib\arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


(4980116, 15)


  0%|                                                                                          | 0/177 [00:00<?, ?it/s]

(1589782, 15)


100%|████████████████████████████████████████████████████████████████████████████████| 177/177 [03:38<00:00,  1.23s/it]


...

,system,zone2,zone3,zone4,zone2_match,zone3_match,zone4_match
648798,8 - 9,4,2,5,5_Zone,3_Zone,1_Zone
648799,8 - 9,3,2,5,4_Zone,3_Zone,1_Zone
648800,8 - 9,4,2,1,5_Zone,3_Zone,2_Zone
648801,8 - 9,3,2,1,4_Zone,3_Zone,2_Zone
648802,8 - 9,4,1,2,5_Zone,2_Zone,3_Zone
648803,8 - 9,3,2,1,4_Zone,3_Zone,2_Zone
648804,8 - 9,0,0,0,NoOtherZone,NoOtherZone,NoOtherZone
648805,8 - 9,3,0,0,4_Zone,NoOtherZone,NoOtherZone
648806,8 - 9,4,1,2,5_Zone,2_Zone,3_Zone
648807,8 - 9,3,1,2,4_Zone,2_Zone,3_Zone


specifically, a small verification of when zone2 was equal to 9


,Timestamp,HenID,Zone,PenID,ts_order,TagID,signalstrength,system,time,date,...,next3_zone,previous3_zone,next4_zone,previous4_zone,next5_zone,previous5_zone,previousZone==NextZone,zone2_match,zone3_match,zone4_match
649280,2020-11-13 02:20:01,hen_61,2_Zone,pen12,110669,tag_163,8,10 - 12,02:20:01,2020-11-13,...,3_Zone,3_Zone,3_Zone,3_Zone,3_Zone,NaN,True,2_Zone,NoOtherZone,NoOtherZone
649356,2020-11-13 02:24:06,hen_61,2_Zone,pen12,110702,tag_163,7,10 - 12,02:24:06,2020-11-13,...,3_Zone,2_Zone,2_Zone,2_Zone,2_Zone,3_Zone,False,2_Zone,NoOtherZone,NoOtherZone
649465,2020-11-13 02:30:23,hen_122,2_Zone,pen12,110760,tag_49,15,10 - 12,02:30:23,2020-11-13,...,3_Zone,3_Zone,3_Zone,3_Zone,3_Zone,3_Zone,False,2_Zone,3_Zone,NoOtherZone


Total running time: 21.30 mn
(1586818, 41)


,Timestamp,HenID,Trackingsystem_Zone,PenID,ts_order,TagID,signalstrength,system,time,date,...,previous4_zone,next5_zone,previous5_zone,previousZone==NextZone,zone2_match,zone3_match,zone4_match,zone3_match_exist,next2zone==Zone,previous2zone==Zone
648798,2020-11-13 00:12:06,hen_19,4_Zone,pen9,143481,tag_94,8,8 - 9,00:12:06,2020-11-13,...,NaN,5_Zone,NaN,False,5_Zone,3_Zone,1_Zone,1,1,0
648799,2020-11-13 00:12:27,hen_19,5_Zone,pen9,143482,tag_94,8,8 - 9,00:12:27,2020-11-13,...,NaN,4_Zone,NaN,True,4_Zone,3_Zone,1_Zone,1,1,0
648800,2020-11-13 00:13:30,hen_19,4_Zone,pen9,143483,tag_94,8,8 - 9,00:13:30,2020-11-13,...,NaN,5_Zone,NaN,True,5_Zone,3_Zone,2_Zone,1,1,1


In [6]:
#download model
model = CatBoostClassifier()
model.load_model(os.path.join(path_extracted_data_cleaning_model,'FINAL_Catboost'));

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Data preprocessing

In [7]:
#for more flexibility we open the file that was saved previously
df = pd.read_csv(os.path.join(path_extracted_data_cleaning_rawvscleaned,'df_records_initial_processed.csv'), sep=';',
                 parse_dates=['Timestamp','date'], dayfirst=True) 
df['is_dur_smaller_60sec'] = df['duration'].map(lambda x: x<60)
display(df['is_dur_smaller_60sec'].value_counts())
df['duration_bounded_mn'] = df['duration'].map(lambda x: min(x,60*60)/60)
df['next_duration_bounded_mn'] = df['next_duration'].map(lambda x: min(x,60*60)/60)
df['previous_duration_bounded_mn'] = df['previous_duration'].map(lambda x: min(x,60*60)/60)
print(df.shape)
df.head(3)

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


True     794574
False    792244
Name: is_dur_smaller_60sec, dtype: int64

(1586818, 45)


,Timestamp,HenID,Trackingsystem_Zone,PenID,ts_order,TagID,signalstrength,system,time,date,...,zone2_match,zone3_match,zone4_match,zone3_match_exist,next2zone==Zone,previous2zone==Zone,is_dur_smaller_60sec,duration_bounded_mn,next_duration_bounded_mn,previous_duration_bounded_mn
0,2020-11-13 00:12:06,hen_19,4_Zone,pen9,143481,tag_94,8,8 - 9,00:12:06,2020-11-13,...,5_Zone,3_Zone,1_Zone,1,1,0,True,0.35,1.05,0.00
1,2020-11-13 00:12:27,hen_19,5_Zone,pen9,143482,tag_94,8,8 - 9,00:12:27,2020-11-13,...,4_Zone,3_Zone,1_Zone,1,1,0,False,1.05,0.35,0.35
2,2020-11-13 00:13:30,hen_19,4_Zone,pen9,143483,tag_94,8,8 - 9,00:13:30,2020-11-13,...,5_Zone,3_Zone,2_Zone,1,1,1,True,0.35,1.50,1.05


### handle nan

In [8]:
#add info on nan values in the initial df, in order to replace at the end all prediction based on features involvig at least 
#one nan, by 1
#Indeed, for records with some nan features, we will predict them a 1 all the time its only 0,05% of the time , i.e. when its 
#first or last transition. Indeed, we dont want to loose any records!
df['nbr_nan'] = df.filter(li_cont_select+li_cat_select, axis=1).isnull().sum(axis=1).tolist()
display(df['nbr_nan'].value_counts())
#fillnan for simplicity, but we will remove those rows predictions anyway
#display BEFORE categoricalfeatures
display(df[li_cat_select].head(3))
df[li_cont_select] = df[li_cont_select].fillna(df[li_cont_select].mean().iloc[0])
#display AFTER categoricalfeatures
df[li_cat_select] = df[li_cat_select].fillna(df[li_cat_select].mode().iloc[0])
#show data
display(df[config.li_cont_select].head(3))
display(df[config.li_cat_select].head(3))

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0    1586226
1        415
2        177
Name: nbr_nan, dtype: int64

,Trackingsystem_Zone,zone2_match,previous1_zone,next1_zone,system
0,4_Zone,5_Zone,NaN,5_Zone,8 - 9
1,5_Zone,4_Zone,4_Zone,4_Zone,8 - 9
2,4_Zone,5_Zone,5_Zone,5_Zone,8 - 9


,signalstrength,signalstzone2,duration_bounded_mn,next_duration_bounded_mn,previous_duration_bounded_mn,next2zone==Zone,previous2zone==Zone,zone3_match_exist
0,8,7,0.35,1.05,0.00,1,0,1
1,8,7,1.05,0.35,0.35,1,0,1
2,8,7,0.35,1.50,1.05,1,1,1


,Trackingsystem_Zone,zone2_match,previous1_zone,next1_zone,system
0,4_Zone,5_Zone,3_Zone,5_Zone,8 - 9
1,5_Zone,4_Zone,4_Zone,4_Zone,8 - 9
2,4_Zone,5_Zone,5_Zone,5_Zone,8 - 9


# Cleaning

In [9]:
if name_method=='ML-method':
    START_TIME = time.perf_counter()
    #predict, and if its wintergarten (i.e. 1_Zone) put =1
    df['prediction'] = df.apply(lambda x: model.predict(pd.DataFrame(x[li_cont_select+\
                                                                             li_cat_select]).transpose())[0] if\
                                      x['Trackingsystem_Zone']!='1_Zone' else 1, axis=1)
    #predict false if <60seconds in WG
    df.loc[(df['is_dur_smaller_60sec'])&(df['Trackingsystem_Zone']=='1_Zone'), 'prediction'] = 0
    #always =1 if there was some nan values
    df.loc[df['nbr_nan']!=0, 'prediction'] = 1
    END_TIME = time.perf_counter()
    print ("Total running time for prediction: %.2f mn" %((END_TIME-START_TIME)/60))

    #show info & save (take TIME to save)
    display(display(df.head(3)))
    display(df['prediction'].value_counts())
    display(df['prediction'].value_counts(normalize=True)) #in testing it was 4,9% of wrong records: 42/(42+821)*100)
    df.filter(['HenID','PenID','system','Trackingsystem_Zone','prediction','Timestamp','date'],
              axis=1).to_csv(os.path.join(path_extracted_data_cleaning_rawvscleaned, id_run+name_method+'_Rawdata&Model.csv'),
                             index=False, sep=';')    
    
elif name_method=='TD-method':
    #same but with TD
    #predict, and if its wintergarten (i.e. 1_Zone) put =1
    df['prediction'] = df['duration'].map(lambda x: 0 if x<60 else 1)
    #always =1 if there was some nan values for better comparison with the other method
    df.loc[df['nbr_nan']!=0, 'prediction'] = 1
    #show info & save (take TIME to save)
    display(display(df.head(3)))
    display(df['prediction'].value_counts())
    display(df['prediction'].value_counts(normalize=True)) #in testing it was 4,9% of wrong records: 42/(42+821)*100)
    df.filter(['HenID','PenID','system','Trackingsystem_Zone','prediction','Timestamp','date'],
              axis=1).to_csv(os.path.join(path_extracted_data_cleaning_rawvscleaned, id_run+name_method+'_Rawdata&Model.csv'),
                             index=False, sep=';')  
    
else:
    print('choose an existing method (ML-method, or TD-method)')
    sys.exit()

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Total running time for prediction: 73.32 mn


,Timestamp,HenID,Trackingsystem_Zone,PenID,ts_order,TagID,signalstrength,system,time,date,...,zone4_match,zone3_match_exist,next2zone==Zone,previous2zone==Zone,is_dur_smaller_60sec,duration_bounded_mn,next_duration_bounded_mn,previous_duration_bounded_mn,nbr_nan,prediction
0,2020-11-13 00:12:06,hen_19,4_Zone,pen9,143481,tag_94,8,8 - 9,00:12:06,2020-11-13,...,1_Zone,1,1,0,True,0.35,1.05,0.00,1,1
1,2020-11-13 00:12:27,hen_19,5_Zone,pen9,143482,tag_94,8,8 - 9,00:12:27,2020-11-13,...,1_Zone,1,1,0,False,1.05,0.35,0.35,0,1
2,2020-11-13 00:13:30,hen_19,4_Zone,pen9,143483,tag_94,8,8 - 9,00:13:30,2020-11-13,...,2_Zone,1,1,1,True,0.35,1.50,1.05,0,0


None

1    1497754
0      89064
Name: prediction, dtype: int64

1    0.943873
0    0.056127
Name: prediction, dtype: float64

# Compute variable

#IF NOT ALREADY DONE
p = glob.glob(os.path.join(path_initial_data, 'Barn 4 Pen*\TagUpdates\log*'))
#print('There is %d log files:\n  %s'%(len(p),'  \n  '.join(p)))
df = preprocessing_Origins(p, config, save=True, dodevice=True)
print(df.shape)
display(df.head(2))
display(df.tail(2))

In [5]:
#for more flexibility we open the file that was saved previously
df = pd.read_csv(os.path.join(path_extracted_data_cleaning_rawvscleaned, id_run+name_method+'_Rawdata&Model.csv'), sep=';', 
                 parse_dates=['Timestamp','date'], dayfirst=True)
df.rename(columns={'Trackingsystem_Zone':'Zone'}, inplace=True)
print(df.shape)
df.head(3)

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(1586818, 7)


,HenID,PenID,system,Zone,prediction,Timestamp,date
0,hen_19,pen9,8 - 9,4_Zone,1,2020-11-13 00:12:06,2020-11-13
1,hen_19,pen9,8 - 9,5_Zone,1,2020-11-13 00:12:27,2020-11-13
2,hen_19,pen9,8 - 9,4_Zone,0,2020-11-13 00:13:30,2020-11-13


In [6]:
############## cleaned dataset
START_TIME = time.perf_counter()
#remove predicted-wrong-records
print(df.shape)
df_ = df[df['prediction']!=0].copy()
print(df_.shape)
#simply rename the tracking system zone into the model zone 
display(df_.head(2))
for p in tqdm.tqdm(df_['PenID'].unique()):
    df_daily = HenDailyVariable_Origins(df_[df_['PenID']==p], config, name_='CLEANING_'+name_method+p, 
                                        timestamp_name='Timestamp', has_cons_equal_zone=True)
    print(df_daily.shape)
    df_daily.head(3)
END_TIME = time.perf_counter()
print ("Total running time: %.2f mn" %((END_TIME-START_TIME)/60))  

#Concatenate all HenVariables csv into one
li_paths_var = glob.glob(os.path.join(path_extracted_data, id_run+'_daily__CLEANING_'+name_method+'*_variables.csv'))
li_df = []
for path_var in tqdm.tqdm(li_paths_var):
    #for being more reproductible, we open the file that was saved from cleaning
    df_ = pd.read_csv(path_var, sep=';',parse_dates=['level', 'FirstTimestamp_1_Zone', 'FirstTimestamp_2_Zone', 
                                                     'FirstTimestamp_3_Zone','FirstTimestamp_4_Zone',
                                                     'FirstTimestamp_5_Zone']) 
    print(df_.shape)
    li_df.append(df_)
df_daily = pd.concat(li_df)
df_daily.to_csv(os.path.join(path_extracted_data, id_run+name_method+'_daily_CLEANING_ALL_variables.csv'), sep=';', index=False)
print(df_daily.shape)
df_daily.head(3)

(1586818, 7)
(1497754, 7)


C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,HenID,PenID,system,Zone,prediction,Timestamp,date
0,hen_19,pen9,8 - 9,4_Zone,1,2020-11-13 00:12:06,2020-11-13
1,hen_19,pen9,8 - 9,5_Zone,1,2020-11-13 00:12:27,2020-11-13


  0%|                                                                                            | 0/8 [00:00<?, ?it/s]C:\Users\camil\Desktop\animals_code\PhD\UTILS.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.sort_values([timestamp_name], inplace=True)
C:\Users\camil\Desktop\animals_code\PhD\UTILS.py:3942: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[timestamp_name] = df[timestamp_name].map(lambda x: dt.datetime(x.year,x.month,x.day,x.hour,x.minute, x.second))


----------------- Create time serie
in this time series there is 22 hens
The initial starting date in over all is: 2020-11-13 00:12:06, and the ending date will be: 2021-01-11 23:49:31
But note that birds may have different ending and starting date which should be taken into account when computing variables
and after ending the last day at midnight : 2020-11-13 00:12:06, and the ending date will be: 2021-01-11 23:59:59
Total running time: 1.04 mn
your time series has 1 seconds between two timestamps
----------------- main night zone and nbr of transitions over night....
----------------- total duration per Zone in seconds....
----------------- first timestamp in each zone per day....
----------------- number of Zone (excluding nan)....
----------------- compute some variables based on a list of zones over a day....
----------------NESTBOX


...

(1166, 125)


100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:23<00:00,  2.91s/it]

(1257, 125)


(9996, 125)


,HenID,level,duration_1_Zone,duration_2_Zone,duration_3_Zone,duration_4_Zone,duration_5_Zone,verification_daily_total_duration,dur_values,dur_values_normalized,...,list_of_night20_2_temperature,nbr_temperature_nnnan,temperature_night20_2_median,temperature_night20_2_max,temperature_night20_2_var,list_of_MovementCounter_day,len_MovementCounter_day,MovementCounter_day_amount_nnactivity,MovementCounter_day_max,MovementCounter_day_mean
0,hen_131,2020-11-13,0.0,15914.0,25666.0,2138.0,9845.0,53563.0,"[0.0, 15914.0, 25666.0, 2138.0, 9845.0]","[0.0, 0.29710807833765845, 0.479174056718257, ...",...,"[28.0, 28.0, 28.0, 28.0, 28.0, 28.0, 28.0, 28....",36.0,28.0,29.0,0.222222,"[3.0, 63.0, 70.0, 39.0, 51.0, 41.0, 30.0, 43.0...",90.0,0.033333,97.0,54.577778
1,hen_131,2020-11-14,405.0,17084.0,23716.0,2512.0,10283.0,54000.0,"[405.0, 17084.0, 23716.0, 2512.0, 10283.0]","[0.0075, 0.31637037037037036, 0.43918518518518...",...,"[28.0, 28.0, 28.0, 28.0, 27.0, 28.0, 27.0, 27....",36.0,26.0,28.0,0.750000,"[14.0, 69.0, 47.0, 79.0, 50.0, 46.0, 64.0, 75....",90.0,0.011111,95.0,59.077778
2,hen_131,2020-11-15,0.0,16576.0,28684.0,2187.0,6553.0,54000.0,"[0.0, 16576.0, 28684.0, 2187.0, 6553.0]","[0.0, 0.30696296296296294, 0.5311851851851852,...",...,"[28.0, 28.0, 28.0, 28.0, 28.0, 28.0, 28.0, 28....",36.0,28.0,29.0,0.098765,"[16.0, 70.0, 57.0, 55.0, 60.0, 65.0, 73.0, 73....",90.0,0.033333,91.0,57.922222


In [7]:
############## raw dataset
#open time series per session and compute the variables for each session (car time series make sence at session level), et en 
#plus des variables tel que running entropy over the whole session ateach last timestamp of each level make sence only at 
#session level
#once to much data, do this per pen!!! with name=pens!!
START_TIME = time.perf_counter()
for p in tqdm.tqdm(df['PenID'].unique()):
    df_daily = HenDailyVariable_Origins(df[df['PenID']==p], config, name_='RAW_'+name_method+p, timestamp_name='Timestamp',
                                       has_cons_equal_zone=True)
    print(df_daily.shape)
    df_daily.head(3)
END_TIME = time.perf_counter()
print ("Total running time: %.2f mn" %((END_TIME-START_TIME)/60))  

#Concatenate all HenVariables csv into one
li_paths_var = glob.glob(os.path.join(path_extracted_data, id_run+'_daily__RAW_'+name_method+'*_variables.csv'))
li_df = []
for path_var in tqdm.tqdm(li_paths_var):
    #for being more reproductible, we open the file that was saved from cleaning
    df_ = pd.read_csv(path_var, sep=';',parse_dates=['level', 'FirstTimestamp_1_Zone', 'FirstTimestamp_2_Zone', 
                                                     'FirstTimestamp_3_Zone','FirstTimestamp_4_Zone',
                                                     'FirstTimestamp_5_Zone']) 
    print(df_.shape)
    li_df.append(df_)
df_daily = pd.concat(li_df)
df_daily.to_csv(os.path.join(path_extracted_data, id_run+name_method+'_daily_RAW_ALL_variables.csv'), sep=';', index=False)
print(df_daily.shape)
df_daily.head(3)

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
  0%|                                                                                            | 0/8 [00:00<?, ?it/s]C:\Users\camil\Desktop\animals_code\PhD\UTILS.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.sort_values([timestamp_name], inplace=True)
C:\Users\camil\Desktop\animals_code\PhD\UTILS.py:3942: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] =

----------------- Create time serie
in this time series there is 22 hens
The initial starting date in over all is: 2020-11-13 00:12:06, and the ending date will be: 2021-01-11 23:49:31
But note that birds may have different ending and starting date which should be taken into account when computing variables
and after ending the last day at midnight : 2020-11-13 00:12:06, and the ending date will be: 2021-01-11 23:59:59
Total running time: 1.06 mn
your time series has 1 seconds between two timestamps
----------------- main night zone and nbr of transitions over night....
----------------- total duration per Zone in seconds....
----------------- first timestamp in each zone per day....
----------------- number of Zone (excluding nan)....
----------------- compute some variables based on a list of zones over a day....
----------------NESTBOX



100%|██████████████████████████████████████████████████████████████████████████████████| 22/22 [00:04<00:00,  5.27it/s]
C:\Users\camil\anaconda3\lib\site-packages\numpy\lib\nanfunctions.py:1366: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


------------ add hen basics info
From the focalBirdinfo, you have 160 ative tags
(1298, 110)
(1197, 112)
-------------- Lets remove unwanted dates at PENS level


All the night variables are:  Index(['HenID', 'night_level', 'night_Max_duration_zones',
       'night_distribution_entropy', 'night_Total_number_transition',
       'is_mvt_night', 'level', 'PenID', 'CLASS', 'R-Pen', 'InitialStartDate',
       '29-09 weight', '10-12 juin weight', 'weight 23-11-2020',
       'weight 04-01-2021', 'weight 01-02-21', 'weight 12-04-21', 'Treatment'],
      dtype='object')
['night_level']
-------------- Lets remove dates of tags when they were not giving deviceupdate regularly


...

(1166, 125)


100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:21<00:00,  2.66s/it]

(1257, 125)


(9996, 125)


,HenID,level,duration_1_Zone,duration_2_Zone,duration_3_Zone,duration_4_Zone,duration_5_Zone,verification_daily_total_duration,dur_values,dur_values_normalized,...,list_of_night20_2_temperature,nbr_temperature_nnnan,temperature_night20_2_median,temperature_night20_2_max,temperature_night20_2_var,list_of_MovementCounter_day,len_MovementCounter_day,MovementCounter_day_amount_nnactivity,MovementCounter_day_max,MovementCounter_day_mean
0,hen_131,2020-11-13,0.0,15863.0,25684.0,2138.0,9878.0,53563.0,"[0.0, 15863.0, 25684.0, 2138.0, 9878.0]","[0.0, 0.2961559285327558, 0.4795101095905756, ...",...,"[28.0, 28.0, 28.0, 28.0, 28.0, 28.0, 28.0, 28....",36.0,28.0,29.0,0.222222,"[3.0, 63.0, 70.0, 39.0, 51.0, 41.0, 30.0, 43.0...",90.0,0.033333,97.0,54.577778
1,hen_131,2020-11-14,412.0,17042.0,23717.0,2512.0,10317.0,54000.0,"[412.0, 17042.0, 23717.0, 2512.0, 10317.0]","[0.0076296296296296294, 0.3155925925925926, 0....",...,"[28.0, 28.0, 28.0, 28.0, 27.0, 28.0, 27.0, 27....",36.0,26.0,28.0,0.750000,"[14.0, 69.0, 47.0, 79.0, 50.0, 46.0, 64.0, 75....",90.0,0.011111,95.0,59.077778
2,hen_131,2020-11-15,0.0,16522.0,28734.0,2187.0,6557.0,54000.0,"[0.0, 16522.0, 28734.0, 2187.0, 6557.0]","[0.0, 0.30596296296296294, 0.5321111111111111,...",...,"[28.0, 28.0, 28.0, 28.0, 28.0, 28.0, 28.0, 28....",36.0,28.0,29.0,0.098765,"[16.0, 70.0, 57.0, 55.0, 60.0, 65.0, 73.0, 73....",90.0,0.033333,91.0,57.922222


In [8]:
#saving both files into one 022 339 39 64.
#for being more reproductible, we open the file that was saved from cleaning
df_c = pd.read_csv(os.path.join(path_extracted_data, id_run+name_method+'_daily_CLEANING_ALL_variables.csv'), sep=';', 
                   parse_dates=['level'], dayfirst=True) 
df_c['type'] = 'cleaning'
df_r = pd.read_csv(os.path.join(path_extracted_data, id_run+name_method+'_daily_RAW_ALL_variables.csv'), sep=';', 
                   parse_dates=['level'], dayfirst=True) 
df_r['type'] = 'raw'
df_var = pd.concat([df_c,df_r])
df_var.to_csv(os.path.join(path_extracted_data_cleaning_rawvscleaned,name_method+'_daily_ALL_variables.csv'), sep=';', 
              index=False)
print(df_r.shape,df_c.shape,df_var.shape)
print(df_var.shape)
df_var.head(3)

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(9996, 126) (9996, 126) (19992, 126)
(19992, 126)


,HenID,level,duration_1_Zone,duration_2_Zone,duration_3_Zone,duration_4_Zone,duration_5_Zone,verification_daily_total_duration,dur_values,dur_values_normalized,...,nbr_temperature_nnnan,temperature_night20_2_median,temperature_night20_2_max,temperature_night20_2_var,list_of_MovementCounter_day,len_MovementCounter_day,MovementCounter_day_amount_nnactivity,MovementCounter_day_max,MovementCounter_day_mean,type
0,hen_131,2020-11-13,0.0,15914.0,25666.0,2138.0,9845.0,53563.0,"[0.0, 15914.0, 25666.0, 2138.0, 9845.0]","[0.0, 0.29710807833765845, 0.479174056718257, ...",...,36.0,28.0,29.0,0.222222,"[3.0, 63.0, 70.0, 39.0, 51.0, 41.0, 30.0, 43.0...",90.0,0.033333,97.0,54.577778,cleaning
1,hen_131,2020-11-14,405.0,17084.0,23716.0,2512.0,10283.0,54000.0,"[405.0, 17084.0, 23716.0, 2512.0, 10283.0]","[0.0075, 0.31637037037037036, 0.43918518518518...",...,36.0,26.0,28.0,0.750000,"[14.0, 69.0, 47.0, 79.0, 50.0, 46.0, 64.0, 75....",90.0,0.011111,95.0,59.077778,cleaning
2,hen_131,2020-11-15,0.0,16576.0,28684.0,2187.0,6553.0,54000.0,"[0.0, 16576.0, 28684.0, 2187.0, 6553.0]","[0.0, 0.30696296296296294, 0.5311851851851852,...",...,36.0,28.0,29.0,0.098765,"[16.0, 70.0, 57.0, 55.0, 60.0, 65.0, 73.0, 73....",90.0,0.033333,91.0,57.922222,cleaning


In [9]:
min(df_var['level'].unique()), max(df_var['level'].unique())

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(numpy.datetime64('2020-11-12T00:00:00.000000000'),
 numpy.datetime64('2021-01-11T00:00:00.000000000'))

In [10]:
print(list(df_var.columns))

['HenID', 'level', 'duration_1_Zone', 'duration_2_Zone', 'duration_3_Zone', 'duration_4_Zone', 'duration_5_Zone', 'verification_daily_total_duration', 'dur_values', 'dur_values_normalized', 'FirstTimestamp_1_Zone', 'FirstTimestamp_2_Zone', 'FirstTimestamp_3_Zone', 'FirstTimestamp_4_Zone', 'FirstTimestamp_5_Zone', 'Total_number_zone', 'list_of_durations', 'list_of_zones', 'Max_duration_zones', 'dico_zone_sortedduration', 'Total_number_transition', 'nbr_stays', 'distribution_entropy', 'vertical_travel_distance', 't_DU_missingZone_mvtPerc', 'li_event_chaoticmvt_z_d', 'nbr_stays_3_Zone', 'nbr_stays_4_Zone', 'nbr_stays_5_Zone', 'nbr_stays_2_Zone', 'nbr_stays_1_Zone', 'Max_duration_zone_4', 'down_missingZone_mvtPerc', 'up_missingZone_mvtPerc', 'mean_duration', 'median_duration', 'max_duration', 'min_duration', 'variance_duration', 'percentile_5', 'percentile_15_duration', 'percentile_85_duration', 'percentile_95_duration', 'median_abs_deviation_duration', 'kurtosis_duration', 'skew_duration'

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
